# Project: Migrating Nigeria Medical Graduates
#### By Olaoluwa Ebiekuraju

#### Project Motivation: 
This work was inspired by the increase number of Nigerian doctors relocating to the United Kingdom and other developed countries.

#### Data Sources: 
The data for the number of medical graduates produce in Nigeria from the Medical and Dental Council of Nigeria (MDCN) 
[website](https://www.mdcn.gov.ng/page/education/accredited-medical-and-dental-schools-in-nigeria-as-at-march-2022). A news by [Punch](https://www.google.com/amp/s/punchng.com/727-nigerian-doctors-migrate-to-uk-in-five-months/%3famp), that about 727 Nigeria medical doctors has migrated to the UK between December 2021 and May 2022 (6 months period).

#### Data Description:
1. Fully Accredited Schools: These are schools that are able to train medical/dental students and produce graduates i.e doctors
2. Partially Accredited Schools: These are schools that are able to train medical/dental students and but have not gotten the final accreditation to produce doctors
3. Quota: maximum number of medical/dental graduates a school can produce per accademic year

                                                                                    --- Data Description By Itodo Israel

#### Objectives:
1. Determine the average number of medical doctors a Nigerian University can produce 
2. Determine the maximum number of medical doctors produced in Nigeria every year.
3. Determine the Percentage of Nigeria Doctors That left for the UK in the last one year.

In [1]:
# load libraries
import pandas as pd 
import requests
from bs4 import BeautifulSoup as bs

# display all contents
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth',1)

## Gathering Data

In [2]:
# load the MDCN page that has the accredited schools
url = 'https://www.mdcn.gov.ng/page/education/accredited-medical-and-dental-schools-in-nigeria-as-at-march-2022'
response = requests.get(url)

# store the page in memory using beautiful soup
accredited_schools = bs(response.content, 'lxml')

In [3]:
# Grabing each table from the page  
tables = accredited_schools.find('div', class_="postcontent nobottommargin clearfix").findAll('table')

first_table = tables[0] # fully accredited medical schools 
second_table = tables[1] # partially accredited medical schools
third_table = tables[2]  # fully accredited dental schools
fourth_table = tables[3]  # partially accredited dental schools

The code below was gotten from [stackover](https://stackoverflow.com/a/58274853)

In [4]:
# A function to help grab all the tables
def tableDataText(table):
    def rowgetDataText(tr, coltag='td'):
        return [td.get_text(strip=True) for td in tr.find_all(coltag)]
    rows = []
    trs = table.find_all('tr')
    for tr in trs:
        rows.append(rowgetDataText(tr, 'td'))
    return rows  

# grab tables content
table1 = tableDataText(first_table)  # fully accredited medical schools 
table2 = tableDataText(second_table) # partially accredited medical schools
table3 = tableDataText(third_table)  # fully accredited dental schools
table4 = tableDataText(fourth_table) # partially accredited dental schools

In [5]:
# Pass table content into dataframe
fully_accredited_meds = pd.DataFrame(table1)  # fully accredited medical schools
partially_accredited_meds = pd.DataFrame(table2) # partially accredited medical schools
fully_accredited_dentals = pd.DataFrame(table3)  # fully accredited dental schools
partially_accredited_dentals = pd.DataFrame(table4) # partially accredited dental schools

In [6]:
# A funtion to help change header and index of the tables
def change_header_index(df):
    new_header = df.iloc[0]
    df = df[1:]
    df.columns = new_header
    df.set_index('S/N', inplace=True)
    return df

# Changing header and index of the tables
fully_accredited_meds = change_header_index(fully_accredited_meds)
partially_accredited_meds = change_header_index(partially_accredited_meds)
fully_accredited_dentals = change_header_index(fully_accredited_dentals)
partially_accredited_dentals = change_header_index(partially_accredited_dentals)

## Inspecting the data

##### Only interested in the fully accredited medical schools

In [7]:
# checking the first five rows
fully_accredited_meds.head()

,Name of Institution,Quota
S/N,,
1,"College of Health Sciences, Abia State University Uturu, Abia State.",120
2,"College of Health Sciences, University of Uyo, Akwa Ibom.",150
3,"College of Health Sciences, Nnamdi Azikiwe University Nnewi, Anambra State.",100
4,"College of Medical Sciences, University of Maiduguri, Borno State.",150
5,"College of Medical Sciences, University of Calabar, Cross - Rivers State.",150


In [8]:
# checking the general properties of the data
fully_accredited_meds.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37 entries, 1 to 37
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Name of Institution  37 non-null     object
 1   Quota                37 non-null     object
dtypes: object(2)
memory usage: 888.0+ bytes


In [9]:
# Number of Univerties that can produce the different Quota
fully_accredited_meds['Quota'].value_counts()

150    10
50     10
100    6 
75     5 
180    3 
60     2 
120    1 
Name: Quota, dtype: int64

#### Observation:
> The Quota datatype is in string instead of integer

## cleaning

In [10]:
# change the Quota datatype to integer
fully_accredited_meds['Quota'] = fully_accredited_meds['Quota'].astype('int')

## Analysis

#### 1. Determine the average number of medical doctors a Nigerian University can produce 

In [11]:
# Average number of medicals students that can graduate from each medical schools
graduate_avg = fully_accredited_meds['Quota'].mean() 
round(graduate_avg)

101

#### 2. Determine the the maximum number of medical doctors produced in Nigeria every year.

In [12]:
# Maximum number of medical doctors Nigeria can produce in year as at today 03/09/2022
fully_accredited_meds['Quota'].sum()

3755

### 3. Percentage of Nigeria Doctors That left for the UK in the last one year

In [13]:
# Total number of doctors that for UK in a year
total_num_leaving = 727*2

total_num_leaving

1454

In [14]:
# Percentage of doctors that left for the UK in 2022
per_leaving = total_num_leaving/fully_accredited_meds['Quota'].sum()*100

per_leaving.round(1)

38.7

## Conclusion:

#### Summary:
Nigeria produces about 3755 medical graduates every year, each  meical school contributing an average of 101 doctors. Approximately 1454 doctors left for the UK alone in the last one year, which is about 38.7% of the total number of doctors produced in Nigeria per year.

#### Limitation:
The data for the number of doctor that left for the UK is just over a 6 months period.